In [1]:
def windowLevel(image, WL):
    WindowCenterList = [150, 50, 30, 500, 500]
    WindowWidthList = [600, 175, 30, 3000, 400]
    WL_types = ['Brain', 'Blood', 'Stroke', 'Bone', 'Skull']

    for i in range(len(WL_types)):
        if WL == WL_types[i]:
            break
    WindowCenter = WindowCenterList[i]
    WindowWidth = WindowWidthList[i]
    image_Volume = image.copy()

    upper_pixel_threshold = WindowCenter + WindowWidth / 2
    lower_pixel_threshold = WindowCenter - WindowWidth / 2
    upper_threshold_indices = image_Volume > upper_pixel_threshold
    lower_threshold_indices = image_Volume < lower_pixel_threshold

    image_Volume[upper_threshold_indices] = upper_pixel_threshold
    image_Volume[lower_threshold_indices] = lower_pixel_threshold
    return image_Volume

In [20]:
from cam_dataset_4 import DVTDataset
import matplotlib.pyplot as plt
import pydicom
import torch
import pylab
import torchvision.models as models
import regex as re
import torch.nn as nn
import numpy as np
from torch.nn import functional as F
import cv2
import skimage
from sklearn.metrics import roc_auc_score, precision_score, recall_score

In [32]:
model = models.densenet161()
print("Loading model onto CUDA")
state_dict = torch.load("densenet161-8d451a50.pth")
pattern = re.compile(
            r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')

for key in list(state_dict.keys()):
    res = pattern.match(key)
    if res:
        new_key = res.group(1) + res.group(2)
        state_dict[new_key] = state_dict[key]
        del state_dict[key]
model.load_state_dict(state_dict)
model.classifier = nn.Linear(2208,2)

model.load_state_dict(torch.load("classification_dropout/model_9.pt",  map_location=torch.device('cpu')))
print("Finished Loading")
model.eval()

features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

model._modules.get('features').register_forward_hook(hook_feature)

params = list(model.parameters())
weight_softmax = np.squeeze(params[-2].data.numpy())

def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    #print(feature_conv.shape)
    print(class_idx)
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

test_dataset = DVTDataset(csv_file = "test_femoral.csv", root_dir = "", device = "cpu")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle = True, num_workers = 0)

for i, data in enumerate(test_loader):
    if i > 5:
        break

    inputs, labels, orig_img, image_path = data['image'], data['label'], data['orig_img'], data['image_path']
    image = pydicom.dcmread(image_path[0])
    orig_inputs = orig_img.numpy()
    orig_inputs = orig_inputs[0,0,:,:]
    inputs = inputs.repeat(1,3,1,1).float()
    #inputs, labels = inputs, labels
    HU = image.RescaleSlope * image.pixel_array + image.RescaleIntercept
    HU = windowLevel(HU, 'Brain')
    arrays = [HU for _ in range(3)]
    HU = np.stack(arrays, axis=2)
    HU = np.uint8(HU)
    #inputs = torch.reshape(inputs, (inputs.shape[1], inputs.shape[1], inputs.shape[0]))
    #plt.figure()
    #plt.imshow(inputs, cmap=pylab.cm.gray)
    #image = pydicom.dcmread(img_path[0])
    #img = image.pixel_array
    orig_inputs = (orig_inputs + 1)*255/2.
    orig_inputs = np.uint8(orig_inputs)
    logit = model(inputs)
    preds = torch.argmax(logit, 1)
    inputs = torch.squeeze(inputs)
    h_x = F.softmax(logit, dim=1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    probs = probs.numpy()
    idx = idx.numpy()
    current_prec = precision_score(labels.cpu().numpy(), preds.cpu().numpy())
    current_recall = recall_score(labels.cpu().numpy(), preds.cpu().numpy())
    print("Current Prec", current_prec)
    print("Current Recall", current_recall)
    print("Equal", current_prec == 1.0)
    if current_prec == 1.0:
        print("correct")

        # generate class activation mapping for the top1 prediction
        CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

        height, width = orig_inputs.shape
        heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
        #img = cv2.applyColorMap(cv2.resize(orig_inputs,(width, height)), cv2.COLORMAP_BONE)
        #img = skimage.color.gray2rgb(img)
        #result = heatmap * 0.3 + img * 0.5
        #HU = cv2.applyColorMap(HU, cv2.COLORMAP_BONE)
        result = heatmap * 0.3 + HU * 0.5
        cv2.imwrite('CAM_correct{}.jpg'.format(i), result)
        #plt.imshow('img',result)
        #plt.show()
    
    else:
        print("wrong")

        # generate class activation mapping for the top1 prediction
        CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])
        
        height, width = orig_inputs.shape
        heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
        #img = cv2.applyColorMap(cv2.resize(orig_inputs,(width, height)), cv2.COLORMAP_BONE)
        #img = skimage.color.gray2rgb(img)
        #result = heatmap * 0.3 + img * 0.5
        #HU = cv2.applyColorMap(HU, cv2.COLORMAP_BONE)
        result = heatmap * 0.3 + HU * 0.5
        cv2.imwrite('CAM_wrong{}.jpg'.format(i), result)
        #plt.imshow('img',result)
        #plt.show()
        

Loading model onto CUDA
Finished Loading
Current Prec 0.0
Current Recall 0.0
Equal False
wrong
[1]
Current Prec 0.0
Current Recall 0.0
Equal False
wrong
[1]
Current Prec 1.0
Current Recall 1.0
Equal True
correct
[1]
Current Prec 0.0
Current Recall 0.0
Equal False
wrong
[1]
Current Prec 0.0
Current Recall 0.0
Equal False
wrong
[1]
Current Prec 1.0
Current Recall 1.0
Equal True
correct
[1]


In [ ]:
test_dataset = DVTDataset(csv_file = "test_femoral.csv", root_dir = "", device = "cpu")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle = True, num_workers = 0)


for i, data in enumerate(test_loader):
    if i > 2:
        break
    inputs, labels, orig_img = data['image'], data['label'], data['orig_img']
    orig_inputs = orig_img.numpy()
    orig_inputs = orig_inputs[0,0,:,:]
    height, width = orig_inputs.shape
    #orig_inputs = cv2.cvtColor(orig_inputs, cv2.COLOR_GRAY2BGR)
    orig_inputs = (orig_inputs + 1)*255/2.
    orig_inputs = np.uint8(orig_inputs)
    heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
    img = cv2.applyColorMap(orig_inputs, cv2.COLORMAP_BONE)
    result = heatmap * 0.3 + img * 0.5
    cv2.imwrite('CAM.jpg', result)